<h3><a>Goal</a></h3>

- Basic ML Implementation
- Linear Regression

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('practice-ml').getOrCreate()

22/08/01 01:40:49 WARN Utils: Your hostname, harshits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.10 instead (on interface en0)
22/08/01 01:40:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/08/01 01:40:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/01 01:40:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/01 01:40:55 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/01 01:40:55 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [6]:
df = spark.read.csv('../../weight-height.csv', header=True, inferSchema=True)

In [8]:
df.show()

+------+----------------+----------------+
|Gender|          Height|          Weight|
+------+----------------+----------------+
|  Male| 73.847017017515|241.893563180437|
|  Male|68.7819040458903|  162.3104725213|
|  Male|74.1101053917849|  212.7408555565|
|  Male|71.7309784033377|220.042470303077|
|  Male|69.8817958611153|206.349800623871|
|  Male|67.2530156878065|152.212155757083|
|  Male|68.7850812516616|183.927888604031|
|  Male|68.3485155115879|167.971110489509|
|  Male| 67.018949662883| 175.92944039571|
|  Male|63.4564939783664|156.399676387112|
|  Male|71.1953822829745|186.604925560358|
|  Male|71.6408051192206|213.741169489411|
|  Male|64.7663291334055|167.127461073476|
|  Male|69.2830700967204|189.446181386738|
|  Male|69.2437322298112|186.434168021239|
|  Male|67.6456197004212|172.186930058117|
|  Male|72.4183166259878|196.028506330482|
|  Male| 63.974325721061| 172.88347020878|
|  Male|69.6400598997523| 185.98395757313|
|  Male|67.9360048540095|182.426648013226|
+------+---

In [9]:
df.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)



In [10]:
df.columns

['Gender', 'Height', 'Weight']

In [15]:
df = df.withColumn('BMI',(703*(df['Weight']/df['Height'])))

In [11]:
from pyspark.ml.feature import VectorAssembler

In [16]:
featureassembler=VectorAssembler(inputCols=["Height","Weight"],outputCol="Independent Features")

In [17]:
output=featureassembler.transform(df)

In [18]:
output.show()

+------+----------------+----------------+------------------+--------------------+
|Gender|          Height|          Weight|               BMI|Independent Features|
+------+----------------+----------------+------------------+--------------------+
|  Male| 73.847017017515|241.893563180437| 2302.749410656825|[73.847017017515,...|
|  Male|68.7819040458903|  162.3104725213|1658.9285185583867|[68.7819040458903...|
|  Male|74.1101053917849|  212.7408555565|2018.0354712165592|[74.1101053917849...|
|  Male|71.7309784033377|220.042470303077| 2156.527905603826|[71.7309784033377...|
|  Male|69.8817958611153|206.349800623871|2075.8469076393617|[69.8817958611153...|
|  Male|67.2530156878065|152.212155757083|1591.0832310324224|[67.2530156878065...|
|  Male|68.7850812516616|183.927888604031| 1879.787060446488|[68.7850812516616...|
|  Male|68.3485155115879|167.971110489509|  1727.67015918736|[68.3485155115879...|
|  Male| 67.018949662883| 175.92944039571| 1845.424275079034|[67.018949662883,...|
|  M

In [19]:
finalized_data=output.select("Independent Features","BMI")

In [20]:
finalized_data.show()

+--------------------+------------------+
|Independent Features|               BMI|
+--------------------+------------------+
|[73.847017017515,...| 2302.749410656825|
|[68.7819040458903...|1658.9285185583867|
|[74.1101053917849...|2018.0354712165592|
|[71.7309784033377...| 2156.527905603826|
|[69.8817958611153...|2075.8469076393617|
|[67.2530156878065...|1591.0832310324224|
|[68.7850812516616...| 1879.787060446488|
|[68.3485155115879...|  1727.67015918736|
|[67.018949662883,...| 1845.424275079034|
|[63.4564939783664...|1732.6669913031053|
|[71.1953822829745...|1842.5810559950112|
|[71.6408051192206...|2097.4086193057383|
|[64.7663291334055...| 1814.069234843414|
|[69.2830700967204...|1922.2685329757219|
|[69.2437322298112...|1892.7809911220374|
|[67.6456197004212...| 1789.434591137356|
|[72.4183166259878...|1902.9445362842846|
|[63.974325721061,...|1899.7789845678842|
|[69.6400598997523...|1877.4642405839663|
|[67.9360048540095...|1887.7461786116346|
+--------------------+------------

In [21]:
from pyspark.ml.regression import LinearRegression

##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])

regressor=LinearRegression(featuresCol='Independent Features', labelCol='BMI')
regressor=regressor.fit(train_data)

22/08/01 01:55:24 WARN Instrumentation: [2db01db9] regParam is zero, which might cause numerical instability and overfitting.
22/08/01 01:55:25 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/08/01 01:55:25 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/08/01 01:55:25 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/08/01 01:55:25 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [22]:
regressor.coefficients

DenseVector([-25.6187, 10.6136])

In [23]:
regressor.intercept

1684.243735932746

In [24]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [25]:
pred_results.predictions.show()

+--------------------+------------------+------------------+
|Independent Features|               BMI|        prediction|
+--------------------+------------------+------------------+
|[55.6682021205121...| 871.1385819351649| 990.2524172150123|
|[55.7397368200335...|1363.6545166183123|1403.8310095270956|
|[56.1053695899164...|1093.8543930707967|1173.4554685199382|
|[56.741741124191,...|1282.8115899155844|1329.5348795934356|
|[56.7644564465812...| 980.5358792298916| 1070.340230114081|
|[56.789386413216,...| 1180.073423403068|1241.1509536884985|
|[56.8222398387379...|1261.6870864817054|1310.9093655148272|
|[56.8560821293767...|1203.8743648963987| 1261.061085534678|
|[56.9751332313244...|1100.2414732924942|1171.0311563099808|
|[57.1658400564674...|1303.6025849125224|1344.8277533239027|
|[57.3130235163555...|1151.4858481727726|1212.3287947383237|
|[57.3139027398949...|1166.9550789908662|1225.7071294243574|
|[57.5535052100887...|  1321.04268010461|1357.6800910771858|
|[57.598029724148,...|11

In [26]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(11.538579927490238, 235.98786561810118)